In [36]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np 
import sys
sys.path.append("..")
import utility.utility_functions as ut 
from sklearn.model_selection import train_test_split


In [37]:
df = ut.load_post_data_prep_all_zone()
df.dropna(inplace=True)
df = df.drop(df[df.trip_time < 5].index)
df = df.drop('pickup_date',axis=1)
df.reset_index(inplace=True,drop=True)

In [38]:
with open('../best_features.txt','r') as file:
    best_features = [line.strip() for line in file]

In [39]:
df

,weekday,time_binned,Tmax,Tmin,Tavg,Tdep,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis,Zone
0,Tuesday,00:00 - 00:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.543171,11.890650,12.167499,41,4
1,Tuesday,01:00 - 01:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.842436,14.737821,11.223936,78,4
2,Tuesday,02:00 - 02:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.235484,11.183333,11.346154,62,4
3,Tuesday,03:00 - 03:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.640308,11.701795,12.651760,65,4
4,Tuesday,04:00 - 04:59,58.0,39.0,48.5,13.3,16.0,0.0,0.06,0.0,0.0,2.635000,11.158889,13.186688,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519912,Tuesday,19:00 - 19:59,45.0,36.0,40.5,5.1,24.0,0.0,0.02,0.0,0.0,1.634367,8.410748,11.789618,245,263
519913,Tuesday,20:00 - 20:59,45.0,36.0,40.5,5.1,24.0,0.0,0.02,0.0,0.0,1.564640,8.290106,11.409292,347,263
519914,Tuesday,21:00 - 21:59,45.0,36.0,40.5,5.1,24.0,0.0,0.02,0.0,0.0,1.736828,8.729789,12.242927,268,263
519915,Tuesday,22:00 - 22:59,45.0,36.0,40.5,5.1,24.0,0.0,0.02,0.0,0.0,1.708687,7.692929,13.436525,198,263


In [40]:
df = ut.get_dummies(['weekday','time_binned'],df)
df = ut.swap_first_and_last_col(df)

In [41]:
y,X = ut.split_X_y(df,'num_of_taxis')

In [42]:
X = X[best_features]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1357)
train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)

In [44]:
x_train_scaler = StandardScaler()
x_test_scaler = StandardScaler()
y_train_scaler = StandardScaler()
y_test_scaler = StandardScaler()

X_train_scaled = x_train_scaler.fit_transform(X_train)
X_test_scaled = x_test_scaler.fit_transform(X_test)
y_train_scaled = y_train_scaler.fit_transform(pd.DataFrame(y_train))
y_test_scaled = y_test_scaler.fit_transform(pd.DataFrame(y_test))

In [45]:
lm = LinearRegression()  
lm.fit(X_train_scaled,y_train_scaled) 

LinearRegression()

In [46]:
fitted_scaled_price = lm.predict(X_train_scaled)
fitted_price = y_train_scaler.inverse_transform(fitted_scaled_price)
predicted_train = pd.Series(fitted_price[:,0],name='predicted_train')

In [47]:
train_df = round(pd.merge(left=train_df, right=predicted_train , left_index=True, right_index=True),ndigits=2)

In [48]:
train_df

,Zone,00:00 - 00:59,01:00 - 01:59,02:00 - 02:59,03:00 - 03:59,04:00 - 04:59,05:00 - 05:59,06:00 - 06:59,07:00 - 07:59,08:00 - 08:59,...,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis,predicted_train
194023,125,0,0,0,0,0,1,0,0,0,...,30.0,0.0,0.62,0.0,0.0,2.20,8.58,13.92,2,116.34
296486,162,0,0,0,0,0,0,0,0,0,...,26.0,0.0,0.00,0.0,0.0,1.32,13.04,6.34,369,150.71
268782,152,0,0,0,0,0,0,0,0,0,...,33.0,0.0,0.00,0.0,0.0,1.33,8.26,9.69,14,189.41
168838,114,0,0,0,0,0,0,0,0,0,...,21.0,0.0,0.50,0.0,0.0,1.68,12.13,8.01,204,131.86
322684,166,0,0,0,0,0,0,0,0,0,...,20.0,0.0,0.00,0.0,0.0,1.75,9.13,11.57,45,199.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28772,24,0,0,0,0,0,0,0,0,0,...,0.0,3.0,0.00,0.0,0.0,1.80,8.54,12.49,16,29.72
6470,4,0,0,0,0,1,0,0,0,0,...,0.0,4.0,0.20,0.0,0.0,1.06,5.22,13.45,2,161.14
239559,143,0,0,0,0,0,0,0,0,0,...,0.0,6.0,0.17,0.0,0.0,1.70,13.00,8.53,159,162.89
57935,45,0,0,0,1,0,0,0,0,0,...,25.0,0.0,0.00,0.0,0.0,2.53,10.83,13.39,22,111.81


In [50]:
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df.num_of_taxis, train_df.predicted_train)))

RMSE: 150.87480625483227


In [51]:
np.std(y)

136.78326951885785